# Cyclistic Analysis: High-Performance SQL with DuckDB

## Project Overview
Most analyses of the Cyclistic dataset rely on standard Pandas workflows, which can struggle with large file sizes (hundreds of MBs). This project demonstrates a **high-performance approach** using **DuckDB**, an in-process SQL OLAP database, to process millions of ride records instantly.

**Key Technical Skills Demonstrated:**
* **DuckDB SQL:** Efficiently querying and aggregating large datasets without memory overhead.
* **Data Cleaning:** Handling nulls, inconsistent timestamps, and outliers directly in SQL.
* **Feature Engineering:** Extracting metrics like `ride_length` and `day_of_week` using vectorized SQL functions.

1. Data Loading & Cleaning

In this step, we begin by importing the raw Divvy trip data for the period November 2024 to October 2025. Since the data comes directly from the public source, it contains inconsistencies such as missing values, incorrect data types, and unprocessed datetime fields. Before performing any analysis, we will clean and prepare the dataset by fixing datetimes, generating new features (like ride length, day of week, and hour of day), removing invalid entries, and standardizing the structure. This ensures that the dataset is accurate, consistent, and ready for detailed analysis in the next sections.

In [1]:
# Install DuckDB (SQL engine)
!pip install duckdb --quiet

# Import essential libraries
import duckdb
import pandas as pd
import os

# The folder path Kaggle created automatically for your private dataset
data_path = '/kaggle/input/cyclistic-bike-share-dataset-nov-2024-oct-2025'

# List all CSV files to confirm access
print("Files found in dataset folder:")
print(os.listdir(data_path))

Files found in dataset folder:
['202505-divvy-tripdata.csv', '202411-divvy-tripdata.csv', '202504-divvy-tripdata.csv', '202509-divvy-tripdata.csv', '202502-divvy-tripdata.csv', '202412-divvy-tripdata.csv', '202506-divvy-tripdata.csv', '202507-divvy-tripdata.csv', '202503-divvy-tripdata.csv', '202501-divvy-tripdata.csv', '202508-divvy-tripdata.csv', '202510-divvy-tripdata.csv']


In [2]:
# Load and preview November 2024 data using DuckDB + Pandas

# Path to one CSV file
file_path = f"{data_path}/202411-divvy-tripdata.csv"

# SQL query: read file and view top 5 rows
query = f"""
SELECT *
FROM read_csv_auto('{file_path}')
LIMIT 5
"""

# Display neatly in a DataFrame
df_preview = duckdb.query(query).to_df()
df_preview

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,578DDD7CE1771FFA,classic_bike,2024-11-07 19:21:58.206,2024-11-07 19:28:57.301,Walsh Park,18067,Leavitt St & North Ave,TA1308000005,41.914610,-87.667968,41.910533,-87.682308,member
1,78B141C50102ABA6,classic_bike,2024-11-22 14:49:00.431,2024-11-22 14:56:15.475,Walsh Park,18067,Leavitt St & Armitage Ave,TA1309000029,41.914610,-87.667968,41.917805,-87.682437,member
2,1E794CF36394E2D7,classic_bike,2024-11-08 09:24:00.238,2024-11-08 09:28:33.480,Walsh Park,18067,Damen Ave & Cortland St,13133,41.914610,-87.667968,41.915983,-87.677335,member
3,E5DD2CAB58D73F98,classic_bike,2024-11-24 17:51:14.144,2024-11-24 18:05:32.574,Clark St & Elm St,TA1307000039,Clark St & Drummond Pl,TA1307000142,41.902973,-87.631280,41.931248,-87.644336,member
4,57F9878BC8C765F1,classic_bike,2024-11-04 14:59:16.032,2024-11-04 15:41:02.456,Clark St & Wellington Ave,TA1307000136,Streeter Dr & Grand Ave,13022,41.936497,-87.647539,41.892278,-87.612043,casual


In [3]:
# Inspect schema of November 2024 file
schema_query = f"""
DESCRIBE SELECT * FROM read_csv_auto('{file_path}');
"""
schema_df = duckdb.query(schema_query).to_df()
schema_df

,column_name,column_type,null,key,default,extra
0,ride_id,VARCHAR,YES,None,None,None
1,rideable_type,VARCHAR,YES,None,None,None
2,started_at,TIMESTAMP,YES,None,None,None
3,ended_at,TIMESTAMP,YES,None,None,None
4,start_station_name,VARCHAR,YES,None,None,None
5,start_station_id,VARCHAR,YES,None,None,None
6,end_station_name,VARCHAR,YES,None,None,None
7,end_station_id,VARCHAR,YES,None,None,None
8,start_lat,DOUBLE,YES,None,None,None
9,start_lng,DOUBLE,YES,None,None,None


In [4]:
# Check December 2024 file
file_path_2 = f"{data_path}/202412-divvy-tripdata.csv"

duckdb.query(f"""
SELECT *
FROM read_csv_auto('{file_path_2}')
LIMIT 5
""").to_df()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,6C960DEB4F78854E,electric_bike,2024-12-31 01:38:35.018,2024-12-31 01:48:45.775,Halsted St & Roscoe St,TA1309000025,Clark St & Winnemac Ave,TA1309000035,41.943632,-87.649083,41.973348,-87.667855,member
1,C0913EEB2834E7A2,classic_bike,2024-12-21 18:41:26.478,2024-12-21 18:47:33.871,Clark St & Wellington Ave,TA1307000136,Halsted St & Roscoe St,TA1309000025,41.936497,-87.647539,41.943632,-87.649083,member
2,848A37DD4723078A,classic_bike,2024-12-21 11:41:01.664,2024-12-21 11:52:45.094,Sheridan Rd & Montrose Ave,TA1307000107,Broadway & Barry Ave,13137,41.961670,-87.654640,41.937582,-87.644098,member
3,3FA09C762ECB48BD,electric_bike,2024-12-26 13:07:27.526,2024-12-26 13:10:54.130,Aberdeen St & Jackson Blvd,13157,Green St & Randolph St*,chargingstx3,41.877726,-87.654787,41.883602,-87.648627,member
4,E60317ADD1A87488,electric_bike,2024-12-13 15:17:55.063,2024-12-13 15:27:32.583,Paulina St & Flournoy St,KA1504000104,Fairfield Ave & Roosevelt Rd,KA1504000102,41.873061,-87.669135,41.866624,-87.694521,member


In [5]:
# Combine all 12 months into one DataFrame
# --------------------------------------------------

# Read every CSV whose name matches the pattern "2024..-divvy-tripdata.csv" or "2025.."
merge_query = f"""
SELECT *
FROM read_csv_auto('{data_path}/2024*-divvy-tripdata.csv')
UNION ALL
SELECT *
FROM read_csv_auto('{data_path}/2025*-divvy-tripdata.csv')
"""

all_trips = duckdb.query(merge_query).to_df()

# Confirm size and columns
print("Combined dataset created successfully!")
print("Total rows:", all_trips.shape[0])
print("Columns:", list(all_trips.columns))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Combined dataset created successfully!
Total rows: 5569279
Columns: ['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual']


In [6]:
# Clean nulls and invalid durations
# -------------------------------------------

clean_query = """
SELECT *
FROM all_trips
WHERE start_station_name IS NOT NULL
  AND end_station_name IS NOT NULL
  AND started_at IS NOT NULL
  AND ended_at IS NOT NULL
  AND started_at < ended_at
"""

clean_df = duckdb.query(clean_query).to_df()

print("Cleaned dataset shape:", clean_df.shape)

Cleaned dataset shape: (3738761, 13)


In [7]:
# Add ride_length, day_of_week, hour_of_day
# ---------------------------------------------------

enhance_query = """
SELECT *,
       ROUND(EXTRACT('epoch' FROM (ended_at - started_at)) / 60, 2) AS ride_length, -- minutes
       strftime(started_at, '%A') AS day_of_week,  -- weekday name
       EXTRACT(hour FROM started_at) AS hour_of_day -- hour number
FROM clean_df
"""

final_df = duckdb.query(enhance_query).to_df()

print("Final dataset with new columns")
final_df.head()

Final dataset with new columns


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,day_of_week,hour_of_day
0,578DDD7CE1771FFA,classic_bike,2024-11-07 19:21:58.206,2024-11-07 19:28:57.301,Walsh Park,18067,Leavitt St & North Ave,TA1308000005,41.914610,-87.667968,41.910533,-87.682308,member,6.98,Thursday,19
1,78B141C50102ABA6,classic_bike,2024-11-22 14:49:00.431,2024-11-22 14:56:15.475,Walsh Park,18067,Leavitt St & Armitage Ave,TA1309000029,41.914610,-87.667968,41.917805,-87.682437,member,7.25,Friday,14
2,1E794CF36394E2D7,classic_bike,2024-11-08 09:24:00.238,2024-11-08 09:28:33.480,Walsh Park,18067,Damen Ave & Cortland St,13133,41.914610,-87.667968,41.915983,-87.677335,member,4.55,Friday,9
3,E5DD2CAB58D73F98,classic_bike,2024-11-24 17:51:14.144,2024-11-24 18:05:32.574,Clark St & Elm St,TA1307000039,Clark St & Drummond Pl,TA1307000142,41.902973,-87.631280,41.931248,-87.644336,member,14.31,Sunday,17
4,57F9878BC8C765F1,classic_bike,2024-11-04 14:59:16.032,2024-11-04 15:41:02.456,Clark St & Wellington Ave,TA1307000136,Streeter Dr & Grand Ave,13022,41.936497,-87.647539,41.892278,-87.612043,casual,41.77,Monday,14


In [8]:
# Simple Direct Summary
# -------------------------------------

import numpy as np
pd.set_option("display.float_format", "{:,.2f}".format)

# Choose only the columns that make business sense
numeric_cols = ['ride_length', 'hour_of_day']  # meaningful numeric metrics
categorical_cols = ['member_casual', 'rideable_type', 'day_of_week']  # useful categories

# Numeric summary
print(" NUMERIC SUMMARY\n" + "-"*40)
display(final_df[numeric_cols].describe().round(2))

# Categorical summary
print("\n CATEGORICAL SUMMARY\n" + "-"*40)
for col in categorical_cols:
    print(f"\n Column: {col}")
    display(final_df[col].value_counts().head(5).to_frame().rename(columns={col: 'Count'}))

 NUMERIC SUMMARY
----------------------------------------


,ride_length,hour_of_day
count,"3,738,761.00","3,738,761.00"
mean,15.81,13.90
std,33.14,4.84
min,0.00,0.00
25%,5.72,10.00
50%,9.85,15.00
75%,17.49,17.00
max,"1,499.64",23.00



 CATEGORICAL SUMMARY
----------------------------------------

 Column: member_casual


,count
member_casual,
member,2401544
casual,1337217



 Column: rideable_type


,count
rideable_type,
classic_bike,2000790
electric_bike,1737971



 Column: day_of_week


,count
day_of_week,
Friday,568415
Saturday,564139
Thursday,546373
Tuesday,538764
Monday,519038


In [9]:
# Check for any negative ride lengths
print("Negative rides:", (final_df['ride_length'] < 0).sum())

Negative rides: 0


In [10]:
#final_df.to_csv('/kaggle/working/trips_clean.csv', index=False, compression='gzip')
print("Compressed CSV (gzip) exported for faster upload!")

Compressed CSV (gzip) exported for faster upload!


2. Beginning the Data Analysis

With the data fully cleaned, validated, and structured, we can now move into the analysis phase of the project. At this stage, the dataset is ready for deeper exploration using both Pandas and DuckDB SQL. Our goal here is to uncover meaningful patterns in rider behavior, such as ride duration trends, member vs. casual differences, peak usage times, and station-level activity. The following queries and visualizations will help us transform the cleaned dataset into actionable insights and data-driven observations.

In [11]:
import pandas as pd
import duckdb

# ----------------------------------------------------------
# Load dataset
# ----------------------------------------------------------
df = pd.read_csv('/kaggle/input/divvy-trips-clean-dataset-nov-2024-oct-2025/trips_clean.csv')

# Create DuckDB connection & register dataframe
con = duckdb.connect()
con.register("trips", df)

In [12]:
# ----------------------------------------------------------
# 1️⃣ Total Rides
# ----------------------------------------------------------
total_rides = con.sql("""
SELECT COUNT(*) AS total_rides FROM trips
""").df()
total_rides

,total_rides
0,3738761


In [13]:
# ----------------------------------------------------------
# 2️⃣ Describe the table
# ----------------------------------------------------------
describe_trips = con.sql("DESCRIBE trips").df()
describe_trips

,column_name,column_type,null,key,default,extra
0,ride_id,VARCHAR,YES,None,None,None
1,rideable_type,VARCHAR,YES,None,None,None
2,started_at,VARCHAR,YES,None,None,None
3,ended_at,VARCHAR,YES,None,None,None
4,start_station_name,VARCHAR,YES,None,None,None
5,start_station_id,VARCHAR,YES,None,None,None
6,end_station_name,VARCHAR,YES,None,None,None
7,end_station_id,VARCHAR,YES,None,None,None
8,start_lat,DOUBLE,YES,None,None,None
9,start_lng,DOUBLE,YES,None,None,None


In [14]:
# ----------------------------------------------------------
# 3️⃣ Null Value Counts
# ----------------------------------------------------------
null_counts = con.sql("""
SELECT 
  SUM(ride_id IS NULL) AS ride_id_nulls,
  SUM(rideable_type IS NULL) AS rideable_type_nulls,
  SUM(started_at IS NULL) AS started_at_nulls,
  SUM(ended_at IS NULL) AS ended_at_nulls,
  SUM(start_station_name IS NULL) AS start_station_nulls,
  SUM(end_station_name IS NULL) AS end_station_nulls,
  SUM(member_casual IS NULL) AS member_type_nulls
FROM trips
""").df()
null_counts

,ride_id_nulls,rideable_type_nulls,started_at_nulls,ended_at_nulls,start_station_nulls,end_station_nulls,member_type_nulls
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [15]:
# ----------------------------------------------------------
# 4️⃣ Average Ride Length by Customer Type
# ----------------------------------------------------------
avg_ride_length = con.sql("""
SELECT 
  member_casual,
  ROUND(AVG(ride_length), 2) AS avg_ride_length_min
FROM trips
GROUP BY member_casual
""").df()
avg_ride_length

,member_casual,avg_ride_length_min
0,casual,22.31
1,member,12.19


In [16]:
# ----------------------------------------------------------
# 5️⃣ Total Rides by Day of Week + Member Type
# ----------------------------------------------------------
rides_day = con.sql("""
SELECT 
  day_of_week,
  member_casual,
  COUNT(*) AS total_rides
FROM trips
GROUP BY day_of_week, member_casual 
ORDER BY 
	CASE day_of_week 
		WHEN 'Monday' THEN 1
		WHEN 'Tuesday' THEN 2
   	    WHEN 'Wednesday' THEN 3
    	WHEN 'Thursday' THEN 4
    	WHEN 'Friday' THEN 5
    	WHEN 'Saturday' THEN 6
    	WHEN 'Sunday' THEN 7
    END
""").df()
rides_day

,day_of_week,member_casual,total_rides
0,Monday,member,359523
1,Monday,casual,159515
2,Tuesday,casual,150478
3,Tuesday,member,388286
4,Wednesday,member,369494
5,Wednesday,casual,143305
6,Thursday,casual,166072
7,Thursday,member,380301
8,Friday,member,355397
9,Friday,casual,213018


In [17]:
# ----------------------------------------------------------
# 6️⃣ Most Popular Day (Members vs Casual)
# ----------------------------------------------------------
top_day = con.sql("""
SELECT 
  member_casual,
  day_of_week,
  total_rides
FROM (
  SELECT 
    member_casual,
    day_of_week,
    COUNT(*) AS total_rides,
    ROW_NUMBER() OVER (
      PARTITION BY member_casual 
      ORDER BY COUNT(*) DESC
    ) AS rank
  FROM trips
  GROUP BY member_casual, day_of_week
)
WHERE rank = 1
""").df()
top_day

,member_casual,day_of_week,total_rides
0,casual,Saturday,275483
1,member,Tuesday,388286


In [18]:
# ----------------------------------------------------------
# 7️⃣ Rides by Hour of Day
# ----------------------------------------------------------
rides_hour = con.sql("""
SELECT 
  hour_of_day,
  member_casual,
  COUNT(*) AS total_rides
FROM trips 
GROUP BY hour_of_day, member_casual 
ORDER BY hour_of_day
""").df()
rides_hour

,hour_of_day,member_casual,total_rides
0,0,member,17633
1,0,casual,22723
2,1,casual,14261
3,1,member,10327
4,2,member,5887
5,2,casual,9114
6,3,casual,4988
7,3,member,3941
8,4,member,5278
9,4,casual,4042


In [19]:
# ----------------------------------------------------------
# 8️⃣ Peak Hour for Each User Type
# ----------------------------------------------------------
peak_hour = con.sql("""
SELECT 
  member_casual,
  hour_of_day,
  total_rides
FROM (
  SELECT 
    member_casual,
    hour_of_day,
    COUNT(*) AS total_rides,
    ROW_NUMBER() OVER (
      PARTITION BY member_casual 
      ORDER BY COUNT(*) DESC
    ) AS rank
  FROM trips
  GROUP BY member_casual, hour_of_day
)
WHERE rank = 1
""").df()
peak_hour

,member_casual,hour_of_day,total_rides
0,casual,17,128050
1,member,17,261815


In [20]:
# ----------------------------------------------------------
# 9️⃣ Top 10 Start Stations
# ----------------------------------------------------------
top_stations = con.sql("""
SELECT 
  member_casual,
  start_station_name,
  total_rides
FROM(
  SELECT 
    member_casual,
    start_station_name,
    COUNT(*) AS total_rides,
    ROW_NUMBER() OVER(
      PARTITION BY member_casual
      ORDER BY COUNT(*) DESC
    ) AS rank
  FROM trips
  GROUP BY member_casual, start_station_name
)
WHERE rank <= 10
""").df()
top_stations

,member_casual,start_station_name,total_rides
0,member,Kingsbury St & Kinzie St,27857
1,member,Clinton St & Washington Blvd,22815
2,member,Clinton St & Madison St,20152
3,member,Clark St & Elm St,19210
4,member,Canal St & Madison St,19174
5,member,Clinton St & Jackson Blvd,16765
6,member,State St & Chicago Ave,16373
7,member,Wells St & Elm St,16035
8,member,Wells St & Concord Ln,15414
9,member,University Ave & 57th St,15149


In [21]:
monthly_trends = con.sql("""
SELECT 
    STRFTIME('%Y-%m', CAST(started_at AS TIMESTAMP)) AS month,
    member_casual,
    COUNT(*) AS total_rides
FROM trips
GROUP BY month, member_casual
ORDER BY month, member_casual
""").df()

monthly_trends


,month,member_casual,total_rides
0,2024-10,casual,7
1,2024-10,member,13
2,2024-11,casual,68802
3,2024-11,member,177130
4,2024-12,casual,28249
5,2024-12,member,102515
6,2025-01,casual,17090
7,2025-01,member,84121
8,2025-02,casual,19631
9,2025-02,member,89945


In [22]:
# ----------------------------------------------------------
# 1️⃣1️⃣ Customer Share %
# ----------------------------------------------------------
percentage_share = con.sql("""
SELECT 
  member_casual,
  COUNT(*) AS total_rides,
  ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS percentage_share
FROM trips
GROUP BY member_casual
""").df()
percentage_share

,member_casual,total_rides,percentage_share
0,member,2401544,64.23
1,casual,1337217,35.77
